**Seleção da matriz A de característima máximo (4x3) e do vetor b (4x1) tais que:**

$$det( [A|b] ) \neq 0 $$

In [468]:
A = random_matrix(ZZ,4,3)
b = random_matrix(ZZ,4,1)
detAb = 0
rankA = 0

while (detAb == 0 or rankA != 3):
    A = random_matrix(ZZ,4,3)
    b = random_matrix(ZZ,4,1)
    detAb = det(block_matrix(1,2,[A, b]))
    rankA = A.rank()


print("Matriz A:")
pretty_print(A)
print(f"car(A) = {rankA}")
print("\nVetor b:")
pretty_print(b)
print(f"\ndet([A|b]) = {detAb}")


Matriz A:


[ 0 33  0]
[-1 -1 -2]
[-7  1  0]
[-1  1 -5]

car(A) = 3

Vetor b:


[  0]
[ -4]
[-10]
[-13]


det([A|b]) = -2376


**Algoritmo de fatorização QR de uma matriz usando Rotações de Givens**

In [469]:
def copy(to_copy):
    l = []
    for line in to_copy:
        l.append(list(line))
    return matrix(l)


def prod_matrizes(L):
    
    result = L[0]
    for i in range(1,len(L)):
        result *= L[i]
    return result
    
    
def rot_givens_fact_QR(A):
    n = A.nrows()
    m = A.ncols()
    R = copy(A)
    In = identity_matrix(RR, n)
    Plist = []
    
    
    for i in range(0,m):
        for j in range(i+1,n):
            cos = R[i,i] / sqrt( (R[i,i]^2) + (R[j,i]^2) )
            sin = R[j,i] / sqrt( (R[i,i]^2) + (R[j,i]^2) )
            P = copy(In)
            P[i,i] = P[j,j] = cos
            P[j,i] = -sin
            P[i,j] = sin
            R = P*R
            Plist.append(P.transpose())
    
    Q = prod_matrizes(Plist)
    
    return Q, R

In [470]:
Q, R = rot_givens_fact_QR(A)

In [471]:
print("Matriz Q:")
pretty_print(Q)
print("\nMatriz R:")
pretty_print(R)


Matriz Q:


[  0.000000000000000   0.999065033867892  0.0116488574479594 -0.0416336669379928]
[ -0.140028008402801 -0.0344300487013296  -0.352204834149885  -0.924747794488108]
[ -0.980196058819607 0.00118724305866654   0.181470327347072  0.0792640966704093]
[ -0.140028008402801  0.0261193472906639  -0.918087457279618   0.369899117795243]


Matriz R:


[     7.14142842854285    -0.980196058819607     0.980196058819607]
[-3.80272600099546e-18      33.0308827566911   -0.0617366390506600]
[-4.15588427051745e-17  1.99979999282469e-17      5.29484695469786]
[-1.02880287285982e-16 -2.47446637114472e-16  2.22044604925031e-16]

**Sabemos que:**

$$ A = QR $$ 

**E queremos calcular x tal que:**

$$ Ax = b $$

**Ou seja, tal que:** 

$$ QRx = b $$

**Visto Q ser uma matriz ortogonal, isto é, a sua inversa é a sua transposta, ao multiplicar ambos os lados da equação, à esquerda, pela transposta de Q, obtém-se:**

$$ Rx = Q^T b $$

Como R é uma matriz triangular superior, resolver esta equação em ordem a x, torna-se relativamente simples ser equivalente a resolver um sistema de equações lineares pelo processo de substituição.

In [472]:
right_side =  (Q.transpose())*b

var('x1', 'x2', 'x3')
x = matrix([[x1], [x2], [x3]])




print("Lado direito do sistema (multiplicação da transposta de Q por b):")
pretty_print(right_side)
print("\nSistema completo:")
pretty_print(R, x, right_side)


Lado direito do sistema (multiplicação da transposta de Q por b):


[  12.1824367310437]
[-0.213703750559977]
[  11.5292530077639]
[ -1.90233832008982]


Sistema completo:


[     7.14142842854285    -0.980196058819607     0.980196058819607]
[-3.80272600099546e-18      33.0308827566911   -0.0617366390506600]
[-4.15588427051745e-17  1.99979999282469e-17      5.29484695469786]
[-1.02880287285982e-16 -2.47446637114472e-16  2.22044604925031e-16] [x1]
[x2]
[x3] [  12.1824367310437]
[-0.213703750559977]
[  11.5292530077639]
[ -1.90233832008982]

**Como a última linha da matriz R é composta apenas por zeros e o último valor do vetor resultante da multiplicação**

$$ Q^Tb $$ 

**é diferente de 0, o sistema é impossível. No entanto, é possível calcular uma aproximação usando o método da substituição para as restantes equações. Essa aproximação tem como objetivo minimizar o valor de** 

$$\| Ax - b \|$$

In [473]:
x3 = right_side[2,0]/R[2,2]
x2 = (right_side[1,0] - R[1,2]*x3)/R[1,1]
x1 = (right_side[0,0] - R[0,2]*x3 - R[0,1]*x2)/R[0,0]
x = matrix([[x1],[x2],[x3]])

In [474]:
print("Solução aproximada:")
pretty_print(x)

Solução aproximada:


[    1.40668754735656]
[-0.00240004000066668]
[    2.17744782925869]

In [475]:
print("Resultado de Ax e de b:")
pretty_print(A*x, b)

Resultado de Ax e de b:


[-0.0792013200220004]
[  -5.75918316587328]
[  -9.84921287149658]
[  -12.2963267336507] [  0]
[ -4]
[-10]
[-13]

In [476]:
import numpy

print("Valor de ||Ax - b||:")
pretty_print(np.linalg.norm(A*x - b))

Valor de ||Ax - b||:


1.902338320089823